# U-Net : Convolutional Networks for Biomedical Image Segmentation

## 0. Abstract

일반적으로 성공적인 DNN 학습을 위해서는 굉장히 많은 주석달린 훈련용 샘플들이 필요하다는 것이 자명하다. 이 논문에서 우리는 data augmentation을 적용해 매우 적은 이미지 데이터로 높은 성능을 내는 네트워크 및 학습 전략을 제안한다. 또한 이 네트워크는 512x512 이미지의 segmentation이 최신 GPU 기준으로 1초도 걸리지 않을 정도로 빠르며, Caffee로 구현한 코드와 학습된 네트워크를 http://lmb.informatik.uni-freiburg.de/people/ronneber/u-net 에서 확인 가능하다.

+ Network Architecture
  + 문맥을 파악하기 위한 **contracting path**
  + 정밀한 localization을 가능하게 하는 **expanding path**
<br><br>
+ Acheivements
  + **ISBI challenge for segmentation of neu-ronal structures in electron microscopic stacks** : 전자현미경으로 얻은 뉴런 이미지들의 구조를 segmentation하는 이 대회에서 기존에 최고의 성능을 내던 sliding-window CNN을 뛰어넘음
  + **2015 ISBI cell tracking challenge** : 투과광선현미경이미지를 학습시켜 큰 차이로 우승

## 1. Introduction

최근 2년 간 DNN은 많은 비전인식분야에서 뛰어난 성능을 내고 있다. 하지만 CNN은 등장한지 오래 되었지만 한계를 맞이하게 되었는데, 이것은 훈련데이터와 네트워크의 크기 때문이다. Krizhevsky et al.의 성공(AlexNet)은 무려 8계층의 큰 네트워크와 100만장의 ImageNet 데이터셋에 적용된 수백만개의 파라미터 덕분이었다. 그 이후로 더 크고 깊은 네트워크들의 학습이 이루어졌다.

CNN의 일반적인 사용처는 단일 클래스로 라벨링된 이미지에 대한 분류 문제이다. 하지만 많은 비전 분야, 특히 바이오 이미지 처리에서는 그 결과로 지역정보를 필요로 한다. 곧 각각의 픽셀마다 라벨링을 한 결과를 필요로 하는 것이다. 게다가 바이오메디컬 분야에서 수많은 이미지를 학습시키는 건 쉬운 일이 아니다. 따라서 Ciresan et al.은 해당 픽셀의 주변 정보를 입력으로 넣어 각각의 픽셀들에 대한 클래스 라벨을 예측하기 위해 sliding-window 네트워크를 제안했다. 먼저, 이 네트워크는 Localization이 가능하다. 그리고 훈련데이터들은 기존의 것보다 더 커지게 된다. 이 네트워크는 큰 차이로 2012 ISBI EM segmentation challenge를 우승했다.

하지만 명백히 그들의 전략은 두 가지 단점이 있다.
+ 네트워크는 각각의 조각마다 따로 학습되어야 하기 때문에 속도가 느리고, 겹치는 조각들 때문에 낭비가 많다. 
+ localization의 정확도와 문맥 사이의 trade-off가 있다 : 큰 조각들은 더 많은 max-pooling layer를 요구하는데, 이러면 localization 정확도가 줄어든다. 반대로 작은 조각들은 문맥 정보를 잘 나타내지 못하게 된다.

이후 연구들은 개선을 통해 localization과 context를 둘 다 좋게 만들어 가고는 있다.




### Fully-Convolutional Network

+ 본 연구 모델의 기반이 되는 네트워크 아키텍쳐

Fully-Convolutional Network의 주 아이디어는 pooling 연산을 upsampling 연산으로 대체하는 것이다. 따라서 각 layer들은 output의 해상도를 높여준다. localization을 위해 contracting path에서 얻어진 고해상도 feature들은 upsample된 output과 결합된다. 이 정보를 기반으로 upsampling 단계에서 convolution layer들은 더 정확한 결과를 만들어낸다.


### Our Modification

+ FCN을 기반으로 무엇을 수정하였는가?

upsampling 단계에서 많은 feature channel들을 갖게 함으로써 네트워크가 문맥 정보를 더 높은 해상도의 레이어에 전달하게 한다. 결과적으로 확장단계는 축소단계와 닮게 되며 U 모양의 구조를 이룬다. 이 네트워크는 fully connected layer가 없으며 각 convolution의 유효한 부분만을 사용한다. 예를 들어 segmentation map은 오직 완전한 문맥 정보만을 갖고 있는 픽셀들만을 담을 뿐이다. 이 전략은 overlap-tile 전략을 통해 임의의 큰 이미지들을 깔끔하게 segmentation시킨다. 이미지 경계부분의 픽셀을 예측하기 위해 소실된 문맥을 입력 이미지를 참고해 추론한다. 이 전략은 큰 이미지를 다루는 네트워크에 중요한 사항이다. 그렇지 않는다면 GPU 메모리 때문에 해상도가 제한될 것이기 때문이다.

### Data Augmentation

사용 가능한 데이터가 적기 때문에 우리는 이미지에 얕은 수정을 가한 data augmentation을 사용했다. 이 작업으로 네트워크는 변형에 강인해진다. 이건 특히 바이오메디컬 segmentation에 중요하다. 이런 변형은 조직에서 가장 흔하게 일어나는데 미리 변형에 대해 효과적으로 시뮬레이트 할 수 있기 때문이다.

### Weighted Loss

세포 segmentation에서 또다른 어려움은 같은 클래스에서 서로 맞닿아있는 물체들을 분리하는 것이다. 해결을 위해 우리는 weighted loss의 사용을 제안한다. 즉 맞닿은 세포들 사이의 background에 loss function 상에서 더 큰 가중치를 부여하는 것이다.

---
## 2. Network Architecture

네트워크 구조는 '축소단계'와 '확장단계'로 나뉘며, 총 23개의 Convolution 레이어로 구성된다.
+ **축소단계**는 CNN의 일반적인 형태와 같다. 각 레이어는 3x3 Convolution + ReLU + 2x2 Max Pooling 으로 구성되어 있다. 우리는 레이어를 거칠 때마다 채널의 수를 두 배로 늘렸다.
+ **확장단계**는 Upsampling을 하는데 2x2 Up-Convolution으로 채널의 수를 반으로 줄이면서 feature map과 합쳐진다. 그리고 3x3 Convolution + ReLU를 2회 씌운다.
+ **마지막 레이어**는 1x1 Convolution으로, 총 64개의 feature vector를 구분하고자 하는 클래스 갯수에 맞춰주며 Segmentation Map을 만들어낸다.

---
## 3. Training

### 학습

### Data Augmentation

사용 가능한 데이터가 적을 때 Data Augmentation은 필수적이다. 현미경 이미지의 경우 데이터의 이동 및 회전에 대한 불변성과 변형 및 그레이스케일의 변화에 대한 강인함이 필요하다. 특히 훈련 데이터의 무작위 변형은 아주 적은 이미지로 네트워크를 학습시킬 수 있는 주요 컨셉이다. 우리는 3x3 무작위 이동 벡터로 약간의 변형을 가했다. 